In [ ]:
test = False
filename = "test.txt" if test else "input.txt"
points = np.loadtxt(filename, dtype=int, delimiter=','  )

In [ ]:
# get the range of the points
x_min, y_min = np.min(points, axis=0)
x_max, y_max = np.max(points, axis=0) 
print(f"x range: {x_min} to {x_max}, y range: {y_min} to {y_max}")

In [ ]:
get_area = lambda p1, p2: (abs(p1[0]-p2[0]) + 1) * (abs(p1[1]-p2[1]) + 1)   
areas = np.zeros((points.shape[0], points.shape[0]), dtype=int)
for i in range(points.shape[0]):
    for j in range(i+1, points.shape[0]):
        area = get_area(points[i], points[j])
        areas[i, j] = area
        areas[j, i] = area
areas
print(f"The biggest area is {np.max(areas)} ")

In [ ]:
# create segments from points that are horizontal vs vertical then the level and range
segments = {'H': [], 'V': []}
for i in range(points.shape[0]):
    p1 = points[i]
    p2 = points[(i+1) % points.shape[0]]
    if p1[0] == p2[0]:  # vertical
        y_start, y_end = sorted([p1[1], p2[1]])
        segments['V'].append((p1[0], (y_start, y_end)))
    elif p1[1] == p2[1]:  # horizontal
        x_start, x_end = sorted([p1[0], p2[0]])
        segments['H'].append((p1[1], (x_start, x_end)))

In [ ]:
# remove areas that are not fully bounded by segments
mod_areas = areas.copy()
for i in range(points.shape[0]-1):
    p1 = points[i]
    for j in range(i+1, points.shape[0]):
        p2 = points[j]
        x_start, x_end = sorted([p1[0], p2[0]])
        y_start, y_end = sorted([p1[1], p2[1]])
        # first, remove areas that have a segment crossing them
        
        # check vertical segments
        if any(x_start < x_level < x_end and not (y_seg_end <= y_start or y_seg_start >= y_end)
               for (x_level, (y_seg_start, y_seg_end)) in segments['V']):
            mod_areas[i, j] = 0
            mod_areas[j, i] = 0
            # print(f"Removing square X: ({x_start}, {x_end}) Y: ({y_start}, {y_end})")
            continue
        # check horizontal segments
        elif any(y_start < y_level < y_end and not (x_seg_end <= x_start or x_seg_start >= x_end)
             for (y_level, (x_seg_start, x_seg_end)) in segments['H']):
            mod_areas[i, j] = 0
            mod_areas[j, i] = 0
            # print(f"Removing square X: ({x_start}, {x_end}) Y: ({y_start}, {y_end})")
            continue
        # second, remove areas that don't have all four sides bounded by segments
        # they don't need to be touching, just bounding. 
        # since we have already removed areas with crossing segments, we can just check a single point on each side

        # check left side
        if not any(x_level <= x_start and y_seg_start <= (y_start + y_end) // 2 <= y_seg_end
                   for (x_level, (y_seg_start, y_seg_end)) in segments['V']):
            mod_areas[i, j] = 0
            mod_areas[j, i] = 0
            # print(f"Removing square X: ({x_start}, {x_end}) Y: ({y_start}, {y_end})")
            continue
        # check right side
        if not any(x_level >= x_end and y_seg_start <= (y_start + y_end) // 2 <= y_seg_end
                   for (x_level, (y_seg_start, y_seg_end)) in segments['V']):
            mod_areas[i, j] = 0
            mod_areas[j, i] = 0
            # print(f"Removing square X: ({x_start}, {x_end}) Y: ({y_start}, {y_end})")
            continue
        # check bottom side
        if not any(y_level <= y_start and x_seg_start <= (x_start + x_end) // 2 <= x_seg_end
                   for (y_level, (x_seg_start, x_seg_end)) in segments['H']):
            mod_areas[i, j] = 0
            mod_areas[j, i] = 0
            # print(f"Removing square X: ({x_start}, {x_end}) Y: ({y_start}, {y_end})")
            continue
        # check top side
        if not any(y_level >= y_end and x_seg_start <= (x_start + x_end) // 2 <= x_seg_end
                   for (y_level, (x_seg_start, x_seg_end)) in segments['H']):
            mod_areas[i, j] = 0
            mod_areas[j, i] = 0
            # print(f"Removing square X: ({x_start}, {x_end}) Y: ({y_start}, {y_end})")
            continue

In [ ]:
ii, jj = np.where(mod_areas == np.max(mod_areas))[0]
print(f"The biggest bounded area is {mod_areas[ii, jj]} from points {points[ii]} and {points[jj]}")